In [ ]:
from init_notebook import *

In [ ]:
image = PIL.Image.open(
    #"/home/bergi/Pictures/WEEK-51-1-euchlid-consortium.png"
    #"/home/bergi/Pictures/__diverse/00-them.jpg"
    #"/home/bergi/Pictures/__diverse/bbjp20250614a0030.jpg"
    "/home/bergi/Pictures/__diverse/Ivy_Lee.jpg"
)
image = image.convert("RGB")
image = VF.to_tensor(image)
image = resize(image, .2)
VF.to_pil_image(image)

In [ ]:
def complexity_map(
        image: torch.Tensor, 
        stride: Union[float, int] = 1./4, 
        fill_value: Union[float, str] = "mean", 
        format: str = "png",
        recursive: int = 0,
        recursive_stride: float = 1./4.,
        verbose: bool = True,
):
    def _get_compressed_size(image: torch.Tensor) -> int:
        fp = io.BytesIO()
        VF.to_pil_image(image).save(fp, format=format)
        return fp.tell()
        
    h, w = image.shape[-2:]
    if isinstance(stride, float):
        stride = int(stride * min(w, h))
    num_y, num_x = (h ) // stride, (w ) // stride
    image = image[:, :num_y * stride, :num_x * stride]
    base_size = _get_compressed_size(image)
    rows = []
    with tqdm(total=num_y * num_x, disable=not verbose) as progress:
        for j in range(num_y):
            row = []
            rows.append(row)
            for i in range(num_x):
                masked = image.clone()
                slice_y = slice(j * stride, (j + 1) * stride)
                slice_x = slice(i * stride, (i + 1) * stride)
                if isinstance(fill_value, float):
                    fill_v = fill_value
                elif fill_value == "mean":
                    fill_v = image[:, slice_y, slice_x].mean(dim=-1).mean(dim=-1)[..., None, None]
                else:
                    raise ValueError(f"Unrecognized fill_value {repr(fill_value)}")
                masked[:, slice_y, slice_x] = fill_v
                size = _get_compressed_size(masked)
                row.append(1. - size / base_size)
                progress.update()

    cm = torch.Tensor(rows)
    if cm.max() == 0:
        print("XX", image.shape, image.min(), image.max())
    cm /= cm.max() + 0.0000001
    
    if recursive > 0:
        rec_rows = []
        with tqdm(total=num_y * num_x, disable=not verbose) as progress:
            for j, orig_row in enumerate(cm):
                row = []
                for i, orig_value in enumerate(orig_row):
                    patch = image[:, j * stride: (j + 1) * stride, i * stride: (i + 1) * stride]
                    row.append(complexity_map(
                        image=patch, 
                        stride=recursive_stride,
                        fill_value=fill_value,
                        format=format,
                        recursive=recursive-1,
                        recursive_stride=recursive_stride,
                        verbose=False,
                    ))
                    progress.update()
                rec_rows.append(torch.concat(row, dim=-1) * orig_value)

        cm = torch.concat(rec_rows, dim=-2)

    if verbose:
        cm_img = (cm - cm.min()) / (cm.max() - cm.min())
        cm_img = .2 + .8 * VF.resize(cm_img.unsqueeze(0), image.shape[-2:], VF.InterpolationMode.NEAREST)
        image2 = image * cm_img#.pow(5)
        display(VF.to_pil_image(image2))

    return cm

cm = complexity_map(image, stride=1./30., recursive=0, recursive_stride=1./4., format="png")
px.imshow(cm)

In [ ]:
cm = complexity_map(image, stride=1./10., recursive=0, recursive_stride=1./4., format="jpeg")
px.imshow(cm)

In [ ]:
def complexity_map_2(
        image: torch.Tensor, 
        stride: Union[float, int] = 1./4, 
        format: str = "png",
        verbose: bool = True,
):
    def _get_compressed_size(image: torch.Tensor) -> int:
        fp = io.BytesIO()
        VF.to_pil_image(image).save(fp, format=format, compression=9)
        return fp.tell()
        
    h, w = image.shape[-2:]
    if isinstance(stride, float):
        stride = int(stride * min(w, h))
    num_y, num_x = (h ) // stride, (w ) // stride
    image = image[:, :num_y * stride, :num_x * stride]
    base_size = _get_compressed_size(image)
    rows = []
    with tqdm(total=num_y * num_x, disable=not verbose) as progress:
        for j in range(num_y):
            row = []
            rows.append(row)
            for i in range(num_x):
                patch = image[:, j * stride: (j + 1) * stride, i * stride: (i + 1) * stride] 
                size = _get_compressed_size(patch)
                row.append(1. - size / base_size)
                progress.update()

    cm = torch.Tensor(rows)
    cm /= cm.max() + 0.0000001    
    return cm

cm = complexity_map_2(image, stride=1./10., format="png")
px.imshow(cm)

In [ ]:
def complexity_map_3(
        image: torch.Tensor, 
        stride: Union[float, int] = 1./4,
        overlap: float = 1./2.,
        fill_value: Union[float, str] = "mean", 
        format: str = "png",
        verbose: bool = True,
):
    def _get_compressed_size(image: torch.Tensor) -> int:
        fp = io.BytesIO()
        VF.to_pil_image(image).save(fp, format=format)
        return fp.tell()
        
    h, w = image.shape[-2:]
    if isinstance(stride, float):
        stride = int(stride * min(w, h))
    num_y, num_x = (h ) // stride, (w ) // stride

    patches = []
    y = 0
    max_y, max_x = 0, 0
    while y + stride <= image.shape[-2]:
        max_y = max(max_y, y)
        x = 0
        while x + stride <= image.shape[-1]:
            max_x = max(max_x, x)
            patches.append({
                "x": x, "y": y,
                "slice_x": slice(x, x + stride),
                "slice_y": slice(y, y + stride),
            })
            x += int(stride * (1. - overlap))
        y += int(stride * (1. - overlap))

    image = image[:, :max_y + stride, :max_x + stride]
    base_size = _get_compressed_size(image)

    cm_img = torch.zeros_like(image)
    cm_count = torch.zeros_like(image)
    for patch in tqdm(patches):
        masked = image.clone()
        slice_x, slice_y = patch["slice_x"], patch["slice_y"]
        if isinstance(fill_value, float):
            fill_v = fill_value
        elif fill_value == "mean":
            fill_v = image[:, slice_y, slice_x].mean(dim=-1).mean(dim=-1)[..., None, None]
        else:
            raise ValueError(f"Unrecognized fill_value {repr(fill_value)}")
        masked[:, slice_y, slice_x] = fill_v
        size = _get_compressed_size(masked)
        cm_img[:, slice_y, slice_x] += size
        cm_count[:, slice_y, slice_x] += 1

    mask = cm_count > 0
    cm_img[mask] /= cm_count[mask]
    cm_img = base_size - cm_img
    print("C", cm_img.shape, cm_img.min(), cm_img.max())
    if verbose:
        cm = (cm_img - cm_img.min()) / (cm_img.max() - cm_img.min())
        cm = .2 + .8 * cm
        image2 = image * cm
        display(VF.to_pil_image(image2))

    return cm_img[0]

cm = complexity_map_3(image, stride=1./10., overlap=1-1/4, format="png")
px.imshow(cm)

In [ ]:
image2 = image[:, :cm.shape[-2], :cm.shape[-1]]
image2 = image2 * (cm / cm.max()).pow(10)
display(VF.to_pil_image(image2))
